### Chatbot with Conversational History

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv('GROQ_API_KEY')

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model='Gemma2-9b-It', groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000014A040D6660>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000014A040D7230>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ('user', 'Hi my name is Yuvraj and Im a GenAI engineer'),
    ("ai", "Hi Yuvraj, its nice to meet you. Thats fascinating, being a GenAI engineer is cutting edge and important work. What kind of projects are you working on? Im always eager to learn more about the advancements in the field.  Perhaps you could tell me about some of the challenges and rewards you encounter in your role?"),
    ('user', "Whats my name and what do i do?")
])

chain=prompt|model
chain.invoke({})

AIMessage(content="You are Yuvraj, and you are a GenAI engineer!  \n\nIs there anything else you'd like to know about yourself or discuss? 😊  I'm here to help!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 108, 'total_tokens': 151, 'completion_time': 0.078181818, 'prompt_time': 0.002806145, 'queue_time': 0.249252525, 'total_time': 0.080987963}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--115c2f7c-e390-497e-ad38-d849d398d11b-0', usage_metadata={'input_tokens': 108, 'output_tokens': 43, 'total_tokens': 151})

### Message History
We can use message history class to wrap our model and make it stateful. This will keep track of the inputs and outputs of the model and store them in some datastore. Future interactions will then load them and pass them into the chain as the input. Lets see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [6]:
config={'configurable': {'session_id': 'chat1'}}

In [7]:
from langchain_core.messages import HumanMessage, AIMessage

response=with_message_history.invoke([
    HumanMessage(content='Hi my name is Yuvraj and Im GenAI Engineer')],
    config=config)

In [8]:
response.content

"Hi Yuvraj, it's nice to meet you! \n\nThat's awesome that you're a GenAI Engineer. It's a really exciting field right now. What kind of work do you do?  \n\nI'm always eager to learn more about how people are using AI to create new and innovative things.\n"

In [9]:
with_message_history.invoke([
    HumanMessage(content='Hi, what is my name and what do i do?')],
    config=config)

AIMessage(content="You told me your name is Yuvraj and that you are a GenAI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 113, 'total_tokens': 155, 'completion_time': 0.076363636, 'prompt_time': 0.002830924, 'queue_time': 0.248333846, 'total_time': 0.07919456}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4b13adc2-83bf-419e-85d3-6920ee765c45-0', usage_metadata={'input_tokens': 113, 'output_tokens': 42, 'total_tokens': 155})

In [10]:
config1 = {'configurable': {'session_id': 'chat2'}}

In [11]:
# changed session id using new config
response=with_message_history.invoke([
    HumanMessage(content='Hi what is my name?')],
    config=config1)
response

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 15, 'total_tokens': 55, 'completion_time': 0.072727273, 'prompt_time': 0.001247111, 'queue_time': 0.247830488, 'total_time': 0.073974384}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a49dce85-7f23-4742-9b79-cca81d81d55b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 40, 'total_tokens': 55})

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions to best of your ability'),
    MessagesPlaceholder(variable_name='messages')
])

chain=prompt|model

In [14]:
chain.invoke({'messages': [HumanMessage(content='Hi my name is Yuvraj')]})

AIMessage(content="Hello Yuvraj, it's nice to meet you! \n\nWhat can I do for you today?  I'm ready to answer your questions and help in any way I can. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 31, 'total_tokens': 77, 'completion_time': 0.083636364, 'prompt_time': 0.001478282, 'queue_time': 0.250632888, 'total_time': 0.085114646}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--5bba0aa7-1dc8-4f3c-8f18-fbdb18446d5d-0', usage_metadata={'input_tokens': 31, 'output_tokens': 46, 'total_tokens': 77})

In [15]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)
config={'configurable': {'session_id': 'chat3'}}

In [17]:
response=with_message_history.invoke([
    HumanMessage(content='Hi my name is yuvraj')],
    config=config
    )
response

AIMessage(content="Hi Yuvraj, it's nice to meet you! \n\nWhat can I do for you today?  I'm ready for any questions you have. 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 31, 'total_tokens': 71, 'completion_time': 0.072727273, 'prompt_time': 0.001498571, 'queue_time': 0.248796638, 'total_time': 0.074225844}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--38f50641-7609-48f8-97bd-5ad63d91ed70-0', usage_metadata={'input_tokens': 31, 'output_tokens': 40, 'total_tokens': 71})

In [18]:
# adds more complexity
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions to the best of your ability in {language}'),
    MessagesPlaceholder(variable_name='messages'),
])
chain = prompt|model

In [20]:
response=chain.invoke({'messages': [HumanMessage(content='Hi My name is Yuvraj')], 'language': 'Hindi'})
response

AIMessage(content='नमस्ते युवराज! \n\nमुझे खुशी है कि तुम मुझसे बात करना चाहते हो। क्या मैं तुम्हारी कोई मदद कर सकता हूँ?   \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 34, 'total_tokens': 76, 'completion_time': 0.076363636, 'prompt_time': 0.001475941, 'queue_time': 0.247590599, 'total_time': 0.077839577}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a8e0cc91-aa28-49c8-952b-4f654fe84b56-0', usage_metadata={'input_tokens': 34, 'output_tokens': 42, 'total_tokens': 76})

Lets now wrap the more complicated chain in MessageHistory class. This time because there are multiple keys in the input we need to specify the correct key to use to save the chat history

In [21]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='messages'
)

In [22]:
config={'configurable': {'session_id': 'chat4'}}
response = with_message_history.invoke({'messages': [HumanMessage(content='Hi my name is Yuvraj')], 'language': 'Hindi'},
                                       config=config)
response.content

'नमस्ते युवराज! \n\nमैं आपकी मदद करने के लिए यहाँ हूँ। \n\nआप मुझसे क्या जानना चाहते हैं? \n'